In [193]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re

from utility import *

In [194]:
BES_data_folder = '../BES_analysis_data/'

In [195]:
Pollbase_folder = create_subdir(BES_data_folder,"Pollbase")
# xl = pd.ExcelFile(Pollbase_folder+'PollBase-Q4-2019.xlsx')
xl = pd.ExcelFile(Pollbase_folder+'PollBase-Q1-2020.xlsx')

In [196]:
# xl.sheet_names

In [197]:
xl.sheet_names

['Introduction',
 '43-45',
 '45-50',
 '50-51',
 '51-55',
 '55-59',
 '59-64',
 '64-66',
 '66-70',
 '70-74',
 '74',
 '74-79',
 '79-83',
 '83-87',
 '87-92',
 '92-97',
 '97-01',
 '01-05',
 '05-10',
 '10-15',
 '15-17',
 '17-19',
 '19-',
 'Monthly average',
 'By election polls',
 '2014 Euros',
 'Gallup ratings 1989-2010',
 'YouGov rolling ratings 2006-7']

In [198]:
df_list = []

In [199]:
def get_fieldwork_days(fw_str, part):
    if pd.isnull(fw_str):
        return np.nan
    elif isinstance(fw_str,int):
        return fw_str
    elif re.match("(\d+|\?)-(\d+|\?)",fw_str):
        temp = [x if x != "?" else np.nan for x in fw_str.split("-") ]
        return temp[part]
    elif re.match("\d+",fw_str):
        return fw_str
    


In [200]:
# Pre-1943 polls
# There were three Gallup polls prior to 1943: Feb 1939 (50% goverment, 44% opposition, 6% don't know), Dec 1939 (54, 30, 16) and Feb 1940 (51, 27, 22)

# 35-43

# Gallup gov/opp/dk
# Con/Lab

# Year,Month,Polling,Con,Lab
# Feb 1939 50, 33,6
# Dec 1939 (54, 30, 16)
# Feb 1940 (51, 27, 22)





In [201]:
sheet_name = xl.sheet_names[1]
print(sheet_name)
df = xl.parse(xl.sheet_names[1])

df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14', 'Unnamed: 15']]
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error',"Notes 1","Notes 2"]


# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)
df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2


df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: x.split("-")[0] if pd.notnull(x) else np.nan).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: x.split("-")[1] if pd.notnull(x) else np.nan).astype("float").astype("Int64") )          

df.drop("Fieldwork",axis=1,inplace=True)




# 8	1945	Jun?	NaN	14/6 and 15/7
df.loc[8,"Month"] = "Jun"
df.loc[8,"Publication Day"] = 14 # 14/15???

# three polls prior to 1943

# Feb 1939 50, 33,6
# Dec 1939 (54, 30, 16)
# Feb 1940 (51, 27, 22)

df.loc[-3,["Year","Month","Polling","Con","Lab","Notes 2","Con lead"]] = ["1939","Feb","Gallup",50,33,"DK 6%",17]
df.loc[-2,["Year","Month","Polling","Con","Lab","Notes 2","Con lead"]] = ["1939","Dec","Gallup",54,30,"DK 16%",24]
df.loc[-1,["Year","Month","Polling","Con","Lab","Notes 2","Con lead"]] = ["1940","Feb","Gallup",51,27,"DK 22%",24]
df = df.sort_index().reset_index(drop=True)

df["Poll_Type"] = "Normal Poll"
# 1945GE = 5 July 1945
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1945,"July",5,5,5,"Result"]

df["Year"] = df["Year"].astype('int')

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df

43-45


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1939,Feb,<NA>,<NA>,<NA>,Gallup,NaN,50.0,NaN,33.0,NaN,NaN,NaN,17.0,NaN,NaN,DK 6%,NaN,NaN,Normal Poll
1,1939,Dec,<NA>,<NA>,<NA>,Gallup,NaN,54.0,NaN,30.0,NaN,NaN,NaN,24.0,NaN,NaN,DK 16%,NaN,NaN,Normal Poll
2,1940,Feb,<NA>,<NA>,<NA>,Gallup,NaN,51.0,NaN,27.0,NaN,NaN,NaN,24.0,NaN,NaN,DK 22%,NaN,NaN,Normal Poll
3,1943,Jun,<NA>,<NA>,<NA>,Gallup,News Chron,36.0,NaN,44.0,NaN,10.5,NaN,-8.0,NaN,NaN,NaN,43-45,2.0,Normal Poll
4,1943,Jul,<NA>,<NA>,<NA>,Gallup,News Chron,33.0,-3.0,47.5,3.5,11.0,0.5,-14.5,-6.5,NaN,NaN,43-45,3.0,Normal Poll
5,1943,Dec,<NA>,<NA>,<NA>,Gallup,News Chron,31.5,-1.5,46.5,-1.0,11.5,0.5,-15.0,-0.5,NaN,NaN,43-45,4.0,Normal Poll
6,1944,Jan,<NA>,<NA>,<NA>,Gallup,News Chron,28.0,-3.5,45.0,-1.5,12.0,0.5,-17.0,-2.0,NaN,NaN,43-45,5.0,Normal Poll
7,1945,Feb,<NA>,<NA>,<NA>,Gallup,News Chron,27.5,-0.5,47.5,2.5,12.5,0.5,-20.0,-3.0,NaN,NaN,43-45,6.0,Normal Poll
8,1945,Apr,<NA>,<NA>,<NA>,Gallup,News Chron,28.0,0.5,47.0,-0.5,14.0,1.5,-19.0,1.0,NaN,NaN,43-45,7.0,Normal Poll
9,1945,May,<NA>,<NA>,<NA>,Gallup,News Chron,33.0,5.0,45.0,-2.0,15.0,1.0,-12.0,7.0,NaN,NaN,43-45,8.0,Normal Poll


In [202]:
# df_final[ df_final["Year"].apply(lambda x: isinstance(x,int)) ]

In [203]:
sheet_name = xl.sheet_names[2]
print(sheet_name)
df = xl.parse(sheet_name)

df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14', 'Unnamed: 15']]
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error',"Notes 1","Notes 2"]
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: x.split("-")[0] if pd.notnull(x) else np.nan).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: x.split("-")[1] if pd.notnull(x) else np.nan).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)



## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1950GE = 23 February 1950
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1950,"Feb",23,23,23,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

45-50


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1946,Jan,<NA>,<NA>,<NA>,Gallup,News Chron,32.0,NaN,52.5,NaN,11.0,NaN,-20.5,NaN,NaN,NaN,45-50,2,Normal Poll
1,1946,May,<NA>,<NA>,<NA>,Gallup,News Chron,40.0,8.0,43.5,-9.0,13.0,2.0,-3.5,17.0,NaN,NaN,45-50,3,Normal Poll
2,1947,Jan,<NA>,<NA>,<NA>,Gallup,News Chron,41.0,1.0,44.5,1.0,12.0,-1.0,-3.5,0.0,NaN,NaN,45-50,4,Normal Poll
3,1947,Mar,<NA>,<NA>,<NA>,Gallup,News Chron,43.5,2.5,43.5,-1.0,10.5,-1.5,0.0,3.5,NaN,NaN,45-50,5,Normal Poll
4,1947,Jun,<NA>,<NA>,<NA>,Gallup,News Chron,42.5,-1.0,42.5,-1.0,12.5,2.0,0.0,0.0,NaN,NaN,45-50,6,Normal Poll
5,1947,Jul,<NA>,<NA>,<NA>,Gallup,News Chron,42.5,0.0,42.5,0.0,12.5,0.0,0.0,0.0,NaN,NaN,45-50,7,Normal Poll
6,1947,Aug,<NA>,<NA>,<NA>,Gallup,News Chron,44.5,2.0,41.0,-1.5,11.0,-1.5,3.5,3.5,NaN,NaN,45-50,8,Normal Poll
7,1947,Sep,<NA>,<NA>,<NA>,Gallup,News Chron,44.5,0.0,39.5,-1.5,11.5,0.5,5.0,1.5,NaN,NaN,45-50,9,Normal Poll
8,1947,Nov,<NA>,<NA>,<NA>,Gallup,News Chron,50.5,6.0,38.0,-1.5,9.0,-2.5,12.5,7.5,NaN,NaN,45-50,10,Normal Poll
9,1948,Jan,<NA>,<NA>,<NA>,Gallup,News Chron,44.5,-6.0,43.5,5.5,10.5,1.5,1.0,-11.5,NaN,NaN,45-50,11,Normal Poll


In [204]:
sheet_name = xl.sheet_names[3]
print(sheet_name)
df = xl.parse(sheet_name)
df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13', 'Unnamed: 14', 'Con good', 'Con bad',
       'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
       'LD net', 'Question', 'Govt good', 'Govt bad',
       'Govt net', 'Question.1']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error', 'Notes 1',
       'Leader: Con good', 'Leader: Con bad',
       'Leader: Con net', 'Leader: Lab good', 'Leader: Lab bad', 'Leader: Lab net',
       'Leader: LD good', 'Leader: LD bad', 'Leader: LD net', 'Leader: Question', 'Govt good', 'Govt bad',
       'Govt net', 'Govt Question']



# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1951GE = 25 October 1951
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1951,"Oct",25,25,25,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

50-51


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,Leader: LD bad,Leader: LD net,Leader: Question,Govt good,Govt bad,Govt net,Govt Question,sheet_name,sheet_row_number,Poll_Type
0,1950,Mar,<NA>,<NA>,<NA>,Gallup,News Chron,43.5,NaN,45.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-51,2,Normal Poll
1,1950,Apr,<NA>,<NA>,<NA>,Gallup,News Chron,45.5,2.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-51,3,Normal Poll
2,1950,May,<NA>,<NA>,<NA>,Gallup,News Chron,43.5,-2.0,46.5,...,NaN,NaN,Do you approve or disapprove of X as Y?,39.0,51.0,-12.0,Are you satisfied or dissatisfied with the Gov...,50-51,4,Normal Poll
3,1950,Jun,<NA>,<NA>,<NA>,Gallup,News Chron,43.5,0.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-51,5,Normal Poll
4,1950,Jul,<NA>,<NA>,<NA>,Gallup,News Chron,42.0,-1.5,43.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-51,6,Normal Poll
5,1950,Aug,<NA>,<NA>,<NA>,Gallup,News Chron,44.5,2.5,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-51,7,Normal Poll
6,1950,Sep,<NA>,<NA>,<NA>,Gallup,News Chron,43.0,-1.5,45.5,...,NaN,NaN,Do you approve or disapprove of X as Y?,44.0,46.0,-2.0,Are you satisfied or dissatisfied with the Gov...,50-51,8,Normal Poll
7,1950,Oct,<NA>,<NA>,<NA>,Gallup,News Chron,42.5,-0.5,45.0,...,NaN,NaN,Do you approve or disapprove of X as Y?,45.0,45.0,0.0,Are you satisfied or dissatisfied with the Gov...,50-51,9,Normal Poll
8,1950,Nov,<NA>,<NA>,<NA>,Gallup,News Chron,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-51,10,Normal Poll
9,1950,Dec,<NA>,<NA>,<NA>,Gallup,News Chron,43.0,NaN,44.0,...,NaN,NaN,Do you approve or disapprove of X as Y?,38.0,51.0,-13.0,Are you satisfied or dissatisfied with the Gov...,50-51,11,Normal Poll


In [205]:
sheet_name = xl.sheet_names[4]
print(sheet_name)
df = xl.parse(sheet_name)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error','Notes 1']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1955GE = 26 May 1955
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1955,"May",26,26,26,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

51-55


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,sheet_name,sheet_row_number,Poll_Type
0,1951,Nov,<NA>,<NA>,<NA>,Gallup,News Chron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Poll conducted but results lost by Gallup,51-55,2,Normal Poll
1,1951,Dec,<NA>,<NA>,<NA>,Gallup,News Chron,47.0,NaN,45.0,NaN,6.5,NaN,2.0,NaN,NaN,51-55,3,Normal Poll
2,1952,Jan,<NA>,<NA>,<NA>,Gallup,News Chron,44.5,-2.5,48.0,3.0,6.0,-0.5,-3.5,-5.5,NaN,51-55,4,Normal Poll
3,1952,Feb,<NA>,<NA>,<NA>,Gallup,News Chron,41.0,-3.5,47.0,-1.0,10.5,4.5,-6.0,-2.5,NaN,51-55,5,Normal Poll
4,1952,Mar,<NA>,<NA>,<NA>,Gallup,News Chron,41.5,0.5,48.0,1.0,9.5,-1.0,-6.5,-0.5,NaN,51-55,6,Normal Poll
5,1952,Apr,<NA>,<NA>,<NA>,Gallup,News Chron,43.5,2.0,49.0,1.0,NaN,NaN,-5.5,NaN,Poll conducted but results lost by Gallup. Res...,51-55,7,Normal Poll
6,1952,May,<NA>,<NA>,<NA>,Gallup,News Chron,43.5,0.0,49.0,0.0,7.0,NaN,-5.5,NaN,NaN,51-55,8,Normal Poll
7,1952,Jun,<NA>,<NA>,<NA>,Gallup,News Chron,40.5,-3.0,49.0,0.0,9.5,2.5,-8.5,-3.0,NaN,51-55,9,Normal Poll
8,1952,Jul,<NA>,<NA>,<NA>,Gallup,News Chron,40.0,-0.5,50.0,1.0,8.5,-1.0,-10.0,-1.5,NaN,51-55,10,Normal Poll
9,1952,Aug,<NA>,<NA>,<NA>,Gallup,News Chron,40.0,0.0,48.0,-2.0,10.5,2.0,-8.0,2.0,NaN,51-55,11,Normal Poll


In [206]:
sheet_name = xl.sheet_names[5]
print(sheet_name)
df = xl.parse(sheet_name)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14','Unnamed: 15']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error', 'Notes 1', 'Notes 2']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1959GE = 8 October 1959
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1959,"Oct",8,8,8,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

55-59


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1955,Jun,<NA>,<NA>,<NA>,Gallup,News Chron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non-Gallup figures recaclulated to exclude ""do...",55-59,2,Normal Poll
1,1955,Jul,2,13,<NA>,Gallup,News Chron,47.0,NaN,43.0,NaN,9.0,NaN,4.0,NaN,NaN,NOP figures only available for Lib + Other,55-59,3,Normal Poll
2,1955,Aug,<NA>,<NA>,<NA>,Gallup,News Chron,44.5,-2.5,47.5,4.5,7.0,-2.0,-3.0,-7.0,NaN,NaN,55-59,4,Normal Poll
3,1955,Sep,9,19,<NA>,Gallup,News Chron,48.0,3.5,44.0,-3.5,7.0,0.0,4.0,7.0,NaN,NaN,55-59,5,Normal Poll
4,1955,Oct,19,26,<NA>,Gallup,News Chron,46.5,-1.5,44.5,0.5,8.0,1.0,2.0,-2.0,NaN,NaN,55-59,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,1959,Oct,3,6,<NA>,Gallup,News Chron,46.5,1.5,47.5,0.5,5.5,-2.0,-1.0,1.0,NaN,NaN,55-59,78,Normal Poll
77,1959,Oct,3,6,8,Gallup,News Chron,48.5,NaN,46.5,NaN,4.5,NaN,2.0,NaN,NaN,NaN,55-59,79,Normal Poll
78,1959,Oct,2,5,8,NOP,DM,48.0,-0.4,44.1,-2.8,7.9,3.2,3.9,2.4,NaN,NaN,55-59,80,Normal Poll
79,1959,Oct,<NA>,<NA>,8,NaN,DE,49.1,-0.6,45.4,-0.8,5.0,1.5,3.7,0.0,NaN,NaN,55-59,81,Normal Poll


In [207]:
sheet_name = xl.sheet_names[6]
print(sheet_name)
df = xl.parse(sheet_name)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error','Notes 1','Notes 2']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1964GE = 15 October 1964
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1964,"Oct",15,15,15,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

59-64


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1959,Oct,<NA>,<NA>,<NA>,Gallup,News Chron,48.0,NaN,46.0,NaN,5.0,NaN,2.0,NaN,NaN,NaN,59-64,2,Normal Poll
1,1959,Nov,<NA>,<NA>,<NA>,Gallup,News Chron,48.0,0.0,44.0,-2.0,7.0,2.0,4.0,2.0,NaN,NaN,59-64,3,Normal Poll
2,1959,Dec,<NA>,<NA>,<NA>,Gallup,News Chron,47.5,-0.5,44.0,0.0,7.5,0.5,3.5,-0.5,NaN,NaN,59-64,4,Normal Poll
3,1960,Jan,1,6,<NA>,Gallup,NaN,47.0,-0.5,43.5,-0.5,8.5,1.0,3.5,0.0,NaN,Gallup switched from News Chronicle to Telegra...,59-64,5,Normal Poll
4,1960,Jan,9,20,<NA>,Gallup,NaN,47.0,0.0,42.5,-1.0,9.5,1.0,4.5,1.0,NaN,NaN,59-64,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,1964,Oct,<NA>,<NA>,15,NaN,DE,44.5,-1.0,43.7,-0.9,11.1,1.7,0.8,-0.1,NaN,NaN,59-64,155,Normal Poll
154,1964,Oct,9,13,15,NOP,DM,44.3,-1.9,47.4,1.1,7.9,0.8,-3.1,-3.0,Reinterview of sample of people previously polled,NaN,59-64,156,Normal Poll
155,1964,Oct,<NA>,<NA>,15,Gallup,DT,42.5,0.0,46.0,-2.5,11.0,2.5,-3.5,2.5,Quota,NaN,59-64,157,Normal Poll
156,1964,Oct,<NA>,<NA>,15,Gallup,NaN,44.5,2.0,46.0,-4.5,9.0,2.5,-1.5,6.5,Random,NaN,59-64,158,Normal Poll


In [208]:
sheet_name = xl.sheet_names[7]
print(sheet_name)
df = xl.parse(sheet_name)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14','Unnamed: 15']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error','Notes 1','Notes 2']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1966GE = 31 March 1966
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1966,"Mar",31,31,31,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

64-66


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1964,Oct,22,25,<NA>,NOP,DM,36.6,NaN,50.7,NaN,12.0,NaN,NaN,NaN,NaN,NaN,64-66,2,Normal Poll
1,1964,Nov,5,8,<NA>,NOP,DM,39.0,2.4,50.3,-0.4,10.5,-1.5,-11.3,NaN,NaN,NaN,64-66,3,Normal Poll
2,1964,Nov,<NA>,<NA>,<NA>,Gallup,DT,38.5,NaN,50.0,NaN,11.0,NaN,-11.5,NaN,NaN,NaN,64-66,4,Normal Poll
3,1964,Nov,19,22,<NA>,NOP,DM,37.9,-1.1,51.7,1.4,10.4,-0.1,-13.8,-2.5,NaN,NaN,64-66,5,Normal Poll
4,1964,Dec,3,7,<NA>,NOP,DM,38.8,0.9,48.7,-3.0,12.2,1.8,-9.9,3.9,NaN,NaN,64-66,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1966,Mar,<NA>,<NA>,29,NaN,DE,40.0,1.0,52.5,-1.0,7.5,0.0,-12.5,2.0,NaN,NaN,64-66,64,Normal Poll
63,1966,Mar,24,28,31,Gallup,DT,40.0,-2.0,51.0,0.5,8.0,1.0,-11.0,-2.5,Combined quota and random,NaN,64-66,65,Normal Poll
64,1966,Mar,27,29,31,NOP,DM,41.6,2.0,50.6,-1.6,7.4,-0.2,-9.0,3.6,NaN,NaN,64-66,66,Normal Poll
65,1966,Mar,<NA>,<NA>,31,NaN,DE,37.4,-2.6,54.1,1.6,7.7,0.2,-16.7,-4.2,NaN,NaN,64-66,67,Normal Poll


In [209]:
sheet_name = xl.sheet_names[8]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14','Unnamed: 15']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error','Notes 1','Notes 2']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1970GE = 18 June 1970
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1970,"Jun",18,18,18,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

66-70


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1966,Apr,14,19,<NA>,Gallup,DT,35.5,NaN,54.5,NaN,8.5,NaN,-19.0,NaN,NaN,NaN,66-70,2,Normal Poll
1,1966,Apr,21,15,<NA>,NOP,DM,35.6,NaN,55.0,NaN,8.7,NaN,-19.4,NaN,NaN,NaN,66-70,3,Normal Poll
2,1966,May,5,9,<NA>,NOP,DM,34.1,-1.5,55.2,0.2,9.6,0.9,-21.1,-1.7,NaN,NaN,66-70,4,Normal Poll
3,1966,May,6,10,<NA>,Gallup,DT,35.5,0.0,53.5,3.2,10.0,3.2,-18.0,3.2,NaN,NaN,66-70,5,Normal Poll
4,1966,Jun,3,7,<NA>,Gallup,DT,39.5,4.0,52.0,-0.7,7.5,-0.7,-12.5,-0.7,NaN,NaN,66-70,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,1970,Jun,12,16,18,NOP,DM,44.0,4.8,48.1,-3.5,6.4,-1.5,-4.1,8.3,NaN,NaN,66-70,188,Normal Poll
187,1970,Jun,13,17,18,ORC,ES,46.5,4.5,45.5,-3.5,6.5,-1.5,1.0,8.0,NaN,Last five polls in order of sampling dates mid...,66-70,189,Normal Poll
188,1970,Jun,14,16,18,Gallup,DT,42.0,-3.5,49.0,1.0,7.5,2.0,-7.0,-4.5,NaN,NaN,66-70,190,Normal Poll
189,1970,Jun,<NA>,<NA>,18,Harris,DE,46.0,1.0,48.0,2.0,5.0,-2.0,-2.0,-1.0,NaN,NaN,66-70,191,Normal Poll


In [210]:
sheet_name = xl.sheet_names[9]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14','Unnamed: 15']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error','Notes 1','Notes 2']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1974GE = 28 February 1974
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1974,"Feb",28,28,28,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

70-74


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1970,Jul,7,13,<NA>,NOP,DM,51.2,NaN,40.4,NaN,6.5,NaN,10.8,NaN,NaN,NaN,70-74,2,Normal Poll
1,1970,Jul,<NA>,<NA>,<NA>,ORC,ES,51.0,NaN,40.0,NaN,8.0,NaN,11.0,NaN,NaN,Butler Political Facts 5th edition inconsisten...,70-74,3,Normal Poll
2,1970,Aug,11,16,<NA>,NOP,DM,47.3,-3.9,44.2,3.8,6.9,0.4,3.1,-7.7,NaN,NaN,70-74,4,Normal Poll
3,1970,Aug,13,16,<NA>,Gallup,DT,47.0,NaN,43.5,NaN,7.5,NaN,3.5,NaN,NaN,NaN,70-74,5,Normal Poll
4,1970,Aug,<NA>,<NA>,<NA>,ORC,ES,49.0,-2.0,41.0,1.0,8.0,0.0,8.0,-3.0,NaN,NaN,70-74,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,1974,Feb,26,27,28,Gallup,DT,39.5,-2.5,37.5,-2.5,20.5,4.0,2.0,0.0,Quota + reinterview,NaN,70-74,178,Normal Poll
177,1974,Feb,27,27,28,ORC,ES,39.7,3.2,36.7,1.7,21.2,-4.3,3.0,1.5,Quota + reinterview,NaN,70-74,179,Normal Poll
178,1974,Feb,<NA>,<NA>,28,ORC,TT,38.7,-1.0,34.9,-1.8,23.0,1.8,3.8,2.8,Quota + reinterview,NaN,70-74,180,Normal Poll
179,1974,Feb,27,27,28,Marplan,Birmingham Evening Mail / LWT,36.5,-2.0,34.5,3.0,25.0,-3.0,2.0,-5.0,Quota + reinterview,NaN,70-74,181,Normal Poll


In [211]:
sheet_name = xl.sheet_names[10]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

# 1974GE2 = 10 October 1974
df.loc[df["Fieldwork"]=="11-16 or 17",["Fieldwork"]]= "11-16"

# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error','Notes 1']

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Poll_Type"] = "Normal Poll"
df.loc[df["Fieldwork"]=="Exit","Poll_Type"]=\
    ["Exit Poll"]

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1974GE2 = 10 October 1974
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1974,"Oct",10,10,10,"Result"]
df.loc[df["Poll_Type"]=="Exit Poll",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1974,"Oct",10,10,10,"Exit Poll"]


df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

74


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,sheet_name,sheet_row_number,Poll_Type
0,1974,Mar,12,17,<NA>,NOP,NaN,35.7,NaN,43.3,NaN,17.9,NaN,-7.6,NaN,NaN,74,2,Normal Poll
1,1974,Mar,13,18,<NA>,Gallup,DT,35.0,NaN,43.0,NaN,19.0,NaN,-8.0,NaN,NaN,74,3,Normal Poll
2,1974,Mar,19,24,<NA>,NOP,NaN,33.6,-2.1,47.7,4.4,15.0,-2.9,-14.1,-6.5,NaN,74,4,Normal Poll
3,1974,Mar,<NA>,<NA>,<NA>,ORC,ES,35.0,NaN,44.0,NaN,17.0,NaN,-9.0,NaN,NaN,74,5,Normal Poll
4,1974,Apr,2,7,<NA>,NOP,NaN,36.4,2.8,45.1,-2.6,16.5,1.5,-8.7,5.4,NaN,74,6,Normal Poll
5,1974,Apr,9,16,<NA>,Gallup,DT,33.0,-2.0,49.0,6.0,15.5,-3.5,-16.0,-8.0,NaN,74,7,Normal Poll
6,1974,Apr,<NA>,<NA>,<NA>,ORC,ES,35.5,0.5,42.5,-1.5,18.0,1.0,-7.0,2.0,NaN,74,8,Normal Poll
7,1974,May,8,13,<NA>,Gallup,DT,33.0,0.0,46.5,-2.5,17.0,1.5,-13.5,2.5,NaN,74,9,Normal Poll
8,1974,May,<NA>,<NA>,<NA>,ORC,ES,35.0,-0.5,45.0,2.5,16.0,-2.0,-10.0,-3.0,NaN,74,10,Normal Poll
9,1974,Jun,12,17,<NA>,Gallup,DT,35.5,2.5,44.0,-2.5,17.0,0.0,-8.5,5.0,NaN,74,11,Normal Poll


In [212]:
sheet_name = xl.sheet_names[11]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

# # 1974GE2 = 10 October 1974
# df.loc[df["Fieldwork"]=="11-16 or 17",["Fieldwork"]]= "11-16"

# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14','Unnamed: 15']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error','Notes 1','Notes 2']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 1979GE = 3 May 1979
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1979,"May",3,3,3,"Result"]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

74-79


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1974,Oct,22,27,<NA>,NOP,DM,36.6,NaN,43.7,NaN,15.1,NaN,-7.1,NaN,NaN,NaN,74-79,2,Normal Poll
1,1974,Oct,<NA>,<NA>,<NA>,ORC,ES,NaN,NaN,NaN,NaN,NaN,NaN,-10.0,NaN,NaN,NaN,74-79,3,Normal Poll
2,1974,Nov,14,18,<NA>,Gallup,DT,35.0,NaN,46.5,NaN,14.5,NaN,-11.5,NaN,NaN,NaN,74-79,4,Normal Poll
3,1974,Nov,26,1,<NA>,NOP,DM,38.4,1.8,47.7,4.0,11.1,-4.0,-9.3,-2.2,NaN,NaN,74-79,5,Normal Poll
4,1974,Nov,<NA>,<NA>,<NA>,ORC,ES,NaN,NaN,NaN,NaN,NaN,NaN,2.0,12.0,NaN,NaN,74-79,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,1979,May,1,2,3,Gallup,DT,43.0,-5,41.0,1.0,13.5,3.0,2.0,-6.0,NaN,NaN,74-79,196,Normal Poll
195,1979,May,2,2,3,MORI,ES,45.0,1,37.0,-4.0,15.0,3.0,8.0,5.0,NaN,NaN,74-79,197,Normal Poll
197,1979,May,2,3,4,Gallup,BBC,45.6,2.6,37.9,-3.1,13.5,0.0,7.7,5.7,NaN,Fieldwork includes polling day,74-79,199,Normal Poll
198,1979,May,3,3,3,ORC,ITN,44.0,NaN,38.0,NaN,14.0,NaN,6.0,NaN,NaN,Fieldwork on polling day,74-79,200,Normal Poll


In [213]:
sheet_name = xl.sheet_names[12]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix
df.loc[df["Fieldwork"]=="28-2 or 11-16",["Fieldwork"]]= "28-2"
df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
df.loc[df["Date"]=="8-9",["Date"]]= "8"



# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13','Unnamed: 14','Unnamed: 15']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error','Notes 1','Notes 2']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Poll_Type"] = "Normal Poll"
df.loc[df["Publisher"]=="ITN",["Poll_Type"]]= "Exit Poll"
df.loc[df["Publisher"]=="BBC",["Poll_Type"]]= "Exit Poll"

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1983GE = 9 June 1983
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1983,"Jun",9,9,9,"Result"]
df.loc[df["Poll_Type"]=="Exit Poll",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1983,"Jun",9,9,9,"Exit Poll"]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

79-83


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1979,Jun,<NA>,7,<NA>,NOP,NaN,44.2,NaN,45.5,NaN,8.2,NaN,-1.3,NaN,NaN,NaN,79-83,2,Normal Poll
1,1979,Jun,13,18,<NA>,Gallup,DT,42.0,NaN,43.5,NaN,12.0,NaN,-1.5,NaN,NaN,NaN,79-83,3,Normal Poll
2,1979,Jul,<NA>,5,<NA>,NOP,NaN,43.1,-1.1,45.4,-0.1,9.5,1.3,-2.3,-1.0,NaN,NaN,79-83,4,Normal Poll
3,1979,Jul,11,16,<NA>,Gallup,DT,41.0,-1.0,46.0,2.5,11.5,-0.5,-5.0,-3.5,NaN,NaN,79-83,5,Normal Poll
4,1979,Aug,8,13,<NA>,Gallup,DT,41.5,0.5,44.0,-2.0,12.5,1.0,-2.5,2.5,NaN,NaN,79-83,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,1983,Jun,8,8,9,NOP,Northcliffe,47.0,1.0,25.0,-3.0,26.0,2.0,22.0,4.0,NaN,NaN,79-83,216,Normal Poll
215,1983,Jun,8,8,9,MORI,ES,44.0,-3.0,28.0,2.0,26.0,1.0,16.0,-5.0,NaN,NaN,79-83,217,Normal Poll
217,1983,Jun,9,9,9,Harris,ITN,42.0,NaN,28.0,NaN,26.0,NaN,14.0,NaN,Actual exit poll,NaN,79-83,219,Exit Poll
218,1983,Jun,9,9,9,Gallup,BBC,43.9,NaN,28.7,NaN,25.8,NaN,15.2,NaN,Not really an exit poll - done through samplin...,NaN,79-83,220,Exit Poll


In [214]:
sheet_name = xl.sheet_names[13]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix
# df.loc[df["Fieldwork"]=="28-2 or 11-16",["Fieldwork"]]= "28-2"
# df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# df.loc[df["Date"]=="8-9",["Date"]]= "8"



# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'SDP',
       'Unnamed: 13', 'Green', 'Unnamed: 15', 'Con lead', 'Unnamed: 17','Unnamed: 18']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'SDP', 'SDP Error', 'Green','Green Error', 'Con lead',
       'Con lead Error','Notes 1']

df.loc[316:317,["Fieldwork"]] = "Exit"
# df.loc[df["Publisher"]=="ITN",["Poll_Type"]]= "Exit Poll"
# df.loc[df["Publisher"]=="BBC",["Poll_Type"]]= "Exit Poll"
# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Poll_Type"] = "Normal Poll"
df.loc[df["Fieldwork"]=="Exit","Poll_Type"]=\
    ["Exit Poll"]


df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
# 1987GE = 11 June 1987
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1987,"Jun",11,11,11,"Result"]
df.loc[df["Poll_Type"]=="Exit Poll",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1987,"Jun",11,11,11,"Exit Poll"]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

83-87


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,SDP,SDP Error,Green,Green Error,Con lead,Con lead Error,Notes 1,sheet_name,sheet_row_number,Poll_Type
0,1983,Jun,16,20,4,MORI,DS,43.0,NaN,27.0,...,NaN,NaN,NaN,NaN,16.0,NaN,NaN,83-87,2,Normal Poll
1,1983,Jul,7,11,13,Gallup,DT,44.0,NaN,28.5,...,NaN,NaN,NaN,NaN,15.5,NaN,NaN,83-87,3,Normal Poll
2,1983,Jul,<NA>,12,<NA>,NOP,NaN,46.4,NaN,27.4,...,NaN,NaN,NaN,NaN,19.0,NaN,NaN,83-87,4,Normal Poll
3,1983,Jul,21,25,1,MORI,DE,44.0,1.0,30.0,...,NaN,NaN,NaN,NaN,14.0,-2.0,NaN,83-87,5,Normal Poll
4,1983,Aug,<NA>,2,<NA>,NOP,NaN,47.1,0.7,24.5,...,NaN,NaN,NaN,NaN,22.6,3.6,NaN,83-87,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,1987,Jun,9,10,11,MORI,TT,44.0,0.0,32.0,...,NaN,NaN,NaN,NaN,12.0,-1.0,NaN,83-87,315,Normal Poll
314,1987,Jun,10,10,<NA>,NOP,Ind,42.0,1.0,35.0,...,NaN,NaN,NaN,NaN,7.0,-1.0,NaN,83-87,316,Normal Poll
316,1987,Jun,11,11,11,NaN,ITN,41.8,NaN,32.0,...,NaN,NaN,NaN,NaN,9.8,NaN,NaN,83-87,318,Exit Poll
317,1987,Jun,11,11,11,NaN,BBC,40.4,NaN,35.4,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,83-87,319,Exit Poll


In [215]:
sheet_name = xl.sheet_names[14]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix
df.loc[df["Fieldwork"]=="24-29?",["Fieldwork"]]= "24-29"
df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
df.loc[df["Fieldwork"]=="2-4?",["Fieldwork"]]= "2-4"
df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
df.loc[df["Date"]=='2?',["Date"]]= '2'
df.loc[df["Date"]=='29?',["Date"]]= '29'
df.loc[562:564,["Fieldwork"]]="Exit"
df.loc[438,["Fieldwork"]] = "30-3"


# df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# df.loc[df["Date"]=="8-9",["Date"]]= "8"



# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'SDP',
       'Unnamed: 13', 'Green', 'Unnamed: 15', 'Con lead', 'Unnamed: 17','Unnamed: 18']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'SDP', 'SDP Error', 'Green','Green Error', 'Con lead',
       'Con lead Error','Notes 1']

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Poll_Type"] = "Normal Poll"
df.loc[df["Fieldwork"]=="Exit","Poll_Type"]=\
    ["Exit Poll"]

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1992GE = 9 April 1992
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1992,"Apr",9,9,9,"Result"]
df.loc[df["Poll_Type"]=="Exit Poll",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1992,"Apr",9,9,9,"Exit Poll"]
df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")

df_list.append(df)
df        

87-92


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,SDP,SDP Error,Green,Green Error,Con lead,Con lead Error,Notes 1,sheet_name,sheet_row_number,Poll_Type
0,1987,Jun,24,29,29,NOP,NaN,42.0,NaN,33.0,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,87-92,2,Normal Poll
1,1987,Jun,25,1,5,MORI,ST,49.0,NaN,31.0,...,NaN,NaN,NaN,NaN,18.0,7.0,NaN,87-92,3,Normal Poll
2,1987,Jul,<NA>,6,<NA>,NOP,NaN,49.0,7.0,31.0,...,NaN,NaN,NaN,NaN,18.0,9.0,NaN,87-92,4,Normal Poll
3,1987,Jul,8,13,19,Gallup,DT,44.5,NaN,33.0,...,NaN,NaN,NaN,NaN,11.5,4.5,NaN,87-92,5,Normal Poll
4,1987,Jul,16,16,19,Harris,OB,48.0,NaN,33.0,...,NaN,NaN,NaN,NaN,15.0,5.0,NaN,87-92,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,1992,Apr,8,8,9,ICM,GU,38.0,1.0,38.0,...,NaN,NaN,NaN,NaN,0.0,2.0,NaN,87-92,562,Normal Poll
562,1992,Apr,9,9,9,ICM,Sun,38.0,NaN,41.0,...,NaN,NaN,NaN,NaN,-3.0,NaN,NaN,87-92,564,Exit Poll
563,1992,Apr,9,9,9,Harris,ITN,41.1,NaN,36.3,...,NaN,NaN,NaN,NaN,4.8,NaN,NaN,87-92,565,Exit Poll
564,1992,Apr,9,9,9,NOP,BBC,40.0,NaN,36.3,...,NaN,NaN,NaN,NaN,3.7,NaN,NaN,87-92,566,Exit Poll


In [216]:
sheet_name = xl.sheet_names[15]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix

df.loc[347:401,"Year"]=np.nan #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

df.loc[df["Fieldwork"]=='25-28?',["Fieldwork"]]= '25-28'
# df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
# df.loc[df["Fieldwork"]=="2-4?",["Fieldwork"]]= "2-4"
# df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
df.loc[df["Date"]=='7?',["Date"]]= '7'
df.loc[df["Date"]=='19??',["Date"]]= '19'
df.loc[405:406,["Fieldwork"]]="Exit"

# # df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# # df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Referendum',
       'Unnamed: 13', 'Con lead', 'Unnamed: 15','Unnamed: 16']]
# # 'Con good', 'Con bad',
# #        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
# #        'LD net', 'Question', 'Govt good', 'Govt bad',
# #        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error',  'Referendum','Referendum Error', 'Con lead',
       'Con lead Error','Notes 1']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Poll_Type"] = "Normal Poll"
df.loc[df["Fieldwork"]=="Exit","Poll_Type"]=\
    ["Exit Poll"]

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
# df["Poll_Type"] = "Normal Poll"
# 1997GE = 1 May 1997
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1997,"May",1,1,1,"Result"]
df.loc[df["Poll_Type"]=="Exit Poll",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [1997,"May",1,1,1,"Exit Poll"]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

92-97


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,LD,LD Error,Referendum,Referendum Error,Con lead,Con lead Error,Notes 1,sheet_name,sheet_row_number,Poll_Type
0,1992,Apr,10,11,<NA>,ASL,NaN,42.0,NaN,36.0,...,18.0,NaN,NaN,NaN,6.0,NaN,Phone,92-97,2,Normal Poll
1,1992,Apr,10,12,<NA>,Gallup,NaN,39.0,NaN,38.0,...,19.0,NaN,NaN,NaN,1.0,NaN,NaN,92-97,3,Normal Poll
2,1992,Apr,25,28,3,MORI,ST,43.0,NaN,38.0,...,16.0,NaN,NaN,NaN,5.0,NaN,NaN,92-97,4,Normal Poll
3,1992,Apr,15,16,19,Harris,OB,44.0,NaN,36.0,...,17.0,NaN,NaN,NaN,8.0,NaN,NaN,92-97,5,Normal Poll
4,1992,Apr,25,28,<NA>,MORI,NaN,39.0,-4.0,37.0,...,18.0,2.0,NaN,NaN,2.0,-3.0,NaN,92-97,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,1997,Apr,30,30,1,MORI,ES,29.0,NaN,46.0,...,19.0,NaN,NaN,NaN,-17.0,3.0,Phone,92-97,403,Normal Poll
403,1997,Apr,<NA>,<NA>,<NA>,G9000,NaN,31.0,1.2,50.0,...,12.6,1.8,NaN,NaN,-19.0,4.9,Final G9000 omnibus poll,92-97,405,Normal Poll
405,1997,May,1,1,1,MORI,ITN,30.0,NaN,46.0,...,18.0,NaN,NaN,NaN,-16.0,NaN,NaN,92-97,407,Exit Poll
406,1997,May,1,1,1,NOP,BBC,29.0,NaN,47.0,...,17.0,NaN,NaN,NaN,-18.0,NaN,NaN,92-97,408,Exit Poll


In [217]:
sheet_name = xl.sheet_names[16]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix

df.loc[df["Date"]=='19?',["Date"]]= '19'




# # df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11',  'Con lead', 'Unnamed: 13','Unnamed: 14','Unnamed: 15']]
# # 'Con good', 'Con bad',
# #        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
# #        'LD net', 'Question', 'Govt good', 'Govt bad',
# #        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error',  'Con lead',
       'Con lead Error', 'Notes 1', 'Notes 2']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Poll_Type"] = "Normal Poll"
df.loc[df["Fieldwork"]=="Exit","Poll_Type"]=\
    ["Exit Poll"]

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
# df["Poll_Type"] = "Normal Poll"
# 2001GE = 7 June 2001
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2001,"Jun",7,7,7,"Result"]
df.loc[df["Poll_Type"]=="Exit Poll",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2001,"Jun",7,7,7,"Exit Poll"]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

97-01


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type
0,1997,May,<NA>,<NA>,<NA>,G9000,DT,23.3,-7.7,56.9,6.9,14.7,2.1,-33.6,-14.6,NaN,NaN,97-01,2,Normal Poll
1,1997,May,29,4,5,Gallup,DT,23.0,-10.0,59.0,13.0,13.0,-3.0,-36.0,-23.0,NaN,Only ICM of the major pollsters regularly used,97-01,3,Normal Poll
2,1997,Jun,6,7,10,ICM,GU,23.0,-10.0,62.0,19.0,14.0,-4.0,-39.0,-29.0,NaN,a prompted voting intention question,97-01,4,Normal Poll
3,1997,Jun,27,28,1,ICM,GU,23.0,0.0,61.0,-1.0,12.0,-2.0,-38.0,1.0,NaN,(ie named Lib Dems) in this Parliament - hence...,97-01,5,Normal Poll
4,1997,Jun,20,23,26,MORI,TT,24.0,-5.0,58.0,12.0,15.0,-4.0,-34.0,-17.0,NaN,NaN,97-01,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,2001,Jun,5,6,7,MORI,TT,30.0,0.0,45.0,-3.0,18.0,2.0,-15.0,3.0,NaN,NaN,97-01,281,Normal Poll
280,2001,Jun,6,6,7,Gallup,DT,32.0,1.0,48.0,1.0,18.0,2.0,-16.0,0.0,NaN,Revised based on late polling,97-01,282,Normal Poll
282,2001,Jun,7,7,7,BBC,NOP,32.0,NaN,44.0,NaN,17.0,NaN,-12.0,NaN,NaN,NaN,97-01,284,Exit Poll
283,2001,Jun,7,7,7,ITN,MORI,31.0,NaN,44.0,NaN,19.0,NaN,-13.0,NaN,NaN,NaN,97-01,285,Exit Poll


In [218]:
sheet_name = xl.sheet_names[17]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


df.loc[df["Fieldwork"]=='8-10?',["Fieldwork"]]= '8-10'
df.loc[df["Published"]=="2-3",["Published"]]= "2"
df.loc[df["Published"]=="3-4",["Published"]]= "3"
df.loc[[67,140],['Unnamed: 15']]= "" #blank the two cells overriding Notes 2 which contain yearly averages




# # # df
df = df[['Year', 'Month', 'Fieldwork', 'Published', 'Polling', 'Publisher',
       'Con', 'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11',
       'Con lead', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Con good', 'Con bad', 'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good',
       'LD bad', 'LD net', 'Question']]




# # # 'Con good', 'Con bad',
# # #        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
# # #        'LD net', 'Question', 'Govt good', 'Govt bad',
# # #        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error',  'Con lead',
       'Con lead Error','Notes 1','Notes 2',
        "Leader: Con good","Leader: Con bad","Leader: Con net",
        "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
        "Leader: LD good", "Leader: LD bad", "Leader: LD net",
        "Leader: Question",
             ]
        # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Poll_Type"] = "Normal Poll"
df.loc[df["Fieldwork"]=="Exit","Poll_Type"]=\
    ["Exit Poll"]

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")
# df["Poll_Type"] = "Normal Poll"
# 2005GE = 5 May 2005
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2005,"May",5,5,5,"Result"]
df.loc[df["Poll_Type"]=="Exit Poll",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2005,"May",5,5,5,"Exit Poll"]
df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

01-05


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,Leader: Lab good,Leader: Lab bad,Leader: Lab net,Leader: LD good,Leader: LD bad,Leader: LD net,Leader: Question,sheet_name,sheet_row_number,Poll_Type
0,2001,Jun,21,26,<NA>,MORI,NaN,25.0,NaN,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,2,Normal Poll
1,2001,Jul,13,14,<NA>,ICM,GU,30.0,NaN,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,3,Normal Poll
2,2001,Jul,19,23,26,MORI,TT,25.0,0.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,4,Normal Poll
3,2001,Jul,20,5,6,ICM,R4,29.0,-1.0,53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,5,Normal Poll
4,2001,Aug,9,14,10,MORI,GMB,23.0,-2.0,53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,6,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2005,May,2,3,5,Populus,TT,32.0,5.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,310,Normal Poll
309,2005,May,3,4,5,YouGov,DT,32.0,0.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,311,Normal Poll
310,2005,May,3,4,5,MORI,ES,33.0,4.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,312,Normal Poll
312,2005,May,5,5,5,NaN,NaN,33.0,NaN,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05,314,Exit Poll


In [219]:
sheet_name = xl.sheet_names[18]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!
df.loc[578:691,"Year"]=np.nan #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

df.loc[df["Fieldwork"]=='15 ?',["Fieldwork"]]= '15'
df.loc[df["Fieldwork"]=="29-1 ?",["Fieldwork"]]= "29-1"
# df.loc[df["Published"]=="3-4",["Published"]]= "3"

df.loc[df["Fieldwork"]=="13-14/11",["Fieldwork"]]= "13-14"
# df.loc[87,["Month"]]= "Aug" ## FIXED!



# # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published', 'Polling',
       'Publisher', 'Con', 'Unnamed: 8', 'M', 'F', 'Net', 'Lab', 'Unnamed: 13',
       'M.1', 'F.1', 'Net.1', 'LD', 'Unnamed: 18', 'M.2', 'F.2', 'Net.2',
       'Con lead', 'Unnamed: 23','Unnamed: 24','Unnamed: 25',
       'Con good','Con bad', 'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good',
       'LD bad', 'LD net', 'Question', ]]
# # # # 'Con good', 'Con bad',
# # # #        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
# # # #        'LD net', 'Question', 'Govt good', 'Govt bad',
# # # #        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day', 'Polling', 'Publisher',
              'Con', 'Con Error', 'Con Male', 'Con Female', 'Con Male-Female',
              'Lab', 'Lab Error', 'Lab Male', 'Lab Female', 'Lab Male-Female',
              'LD', 'LD Error', 'LD Male', 'LD Female', 'LD Male-Female',
              'Con lead', 'Con lead Error','Notes 1','Notes 2',
        "Leader: Con good","Leader: Con bad","Leader: Con net",
        "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
        "Leader: LD good", "Leader: LD bad", "Leader: LD net",
        "Leader: Question",
             ]
#         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df["Fieldwork_Start_check_date"] = df["Unnamed: 3"]
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 2010GE = 6 May 2010
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2010,"May",6,6,6,"Result"]
# df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

05-10


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Con Male,...,Leader: Lab bad,Leader: Lab net,Leader: LD good,Leader: LD bad,Leader: LD net,Leader: Question,sheet_name,sheet_row_number,Fieldwork_Start_check_date,Poll_Type
0,2005,May,19.0,23,<NA>,MORI,First Tuesday,27.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,05-10,2,2005-05-19,Normal Poll
1,2005,May,19.0,23,<NA>,MORI,FT,30.0,NaN,NaN,...,55.0,-16.0,49.0,28.0,21.0,Are you satisfied or dissatisfied with the way...,05-10,3,2005-05-19,Normal Poll
2,2005,May,24.0,26,<NA>,YouGov,DT,31.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,05-10,4,2005-05-24,Normal Poll
3,2005,Jun,16.0,20,7,MORI,Social Research Institute,29.0,-1.0,NaN,...,52.0,-13.0,46.0,26.0,20.0,Are you satisfied or dissatisfied with the way...,05-10,5,2005-06-16,Normal Poll
4,2005,Jun,17.0,19,21,ICM,GU,31.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,05-10,6,2005-06-17,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,2010,May,4.0,5,6,YouGov,Sun,35.0,0.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,05-10,688,2010-05-04,Normal Poll
687,2010,May,4.0,5,6,Opinium,Express,35.0,2.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,05-10,689,2010-05-04,Normal Poll
688,2010,May,4.0,5,6,ComRes,ITV/Ind,37.0,0.0,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,05-10,690,2010-05-04,Normal Poll
689,2010,May,5.0,5,6,MORI,ES,36.0,0.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,05-10,691,2010-05-05,Normal Poll


In [220]:
sheet_name = xl.sheet_names[19]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
import datetime
## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


# Ashcroft - did the fieldwork really take over a month?
df.loc[df["Fieldwork"]=='1-5 Sept',["Fieldwork"]]= '1-5'
# df.loc[df["Fieldwork"]=="29-1 ?",["Fieldwork"]]= "29-1"
df.loc[df["Published"]=="20?",["Published"]]= "20"

df.loc[df["Published"]=="23/24",["Published"]]= "23"
df.loc[df["Published"]==datetime.datetime(2011, 9, 25, 0, 0),["Published"]]= "25"
df.loc[df["Published"]==datetime.datetime(2012, 11, 21, 0, 0),["Published"]]= "21"
df.loc[df["Published"]==datetime.datetime(2014, 1, 4, 0, 0),["Published"]]= "4"
df.loc[df["Published"]==datetime.datetime(2013, 12, 26, 0, 0),["Published"]]= "26"

#(32)	(29)	(Will do…)
# df.loc[112,['Lab good', 'Lab bad', 'Lab net']] = [32,29,3] FIXED IN 2020 RELEASE
# df.loc[116,['Lab good', 'Lab bad', 'Lab net']] = [43,23,20] FIXED IN 2020 RELEASE





# # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published', 'Polling',
       'Publisher', 'Con', 'Unnamed: 8', 'Lab', 'Unnamed: 10', 'LD',
       'Unnamed: 12', 'Con lead', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'UKIP', 'Green', 'BNP', 'Con good', 'Con bad',
       'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
       'LD net', 'UKIP good', 'UKIP bad', 'UKIP net', 'Grn good', 'Grn bad',
       'Grn net', 'Question', 'Con Best PM', 'Lab Best PM', 'LD Best PM',
       'UKIP Best PM', 'Govt good', 'Govt bad', 'Govt net',
       'Question.1', 'Con.1', 'Lab.1', 'LD.1', 'UKIP.1', 'Unnamed: 51',
       ]]



df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day', 'Polling', 'Publisher',
              'Con', 'Con Error',
              'Lab', 'Lab Error',
              'LD', 'LD Error',
              'Con lead', 'Con lead Error','Notes 1','Notes 2',
              'UKIP', 'Green', 'BNP',
            "Leader: Con good","Leader: Con bad","Leader: Con net",
            "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
            "Leader: LD good", "Leader: LD bad", "Leader: LD net",
            "Leader: UKIP good","Leader: UKIP bad","Leader: UKIP net",
            "Leader: Grn good","Leader: Grn bad","Leader: Grn net",
            "Leader: Question",
            "Con Best PM","Lab Best PM","LD Best PM","UKIP Best PM",
            'Govt good', 'Govt bad',  'Govt net', 'Govt Question',
            "Pred Party Share: Con","Pred Party Share: Lab","Pred Party Share: LD","Pred Party Share: UKIP", "Pred Party Share: Question",

             ]
# #         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Poll_Type"] = "Normal Poll"
df.loc[df["Polling"]=="Exit poll","Poll_Type"]=\
    ["Exit Poll"]

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df["Fieldwork_Start_check_date"] = df["Unnamed: 3"]
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")
# df["Poll_Type"] = "Normal Poll"
# 2015GE = 7 May 2015
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2015,"May",7,7,7,"Result"]
df.loc[df["Poll_Type"]=="Exit Poll",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2015,"May",7,7,7,"Exit Poll"]
# df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

10-15


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,Govt Question,Pred Party Share: Con,Pred Party Share: Lab,Pred Party Share: LD,Pred Party Share: UKIP,Pred Party Share: Question,sheet_name,sheet_row_number,Poll_Type,Fieldwork_Start_check_date
0,2010,May,12.0,13,16,ComRes,IoS/SM,38.0,NaN,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10-15,2,Normal Poll,2010-05-12
1,2010,May,12.0,13,16,ICM,STel,38.0,NaN,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10-15,3,Normal Poll,2010-05-12
2,2010,May,13.0,14,16,YouGov,ST,37.0,NaN,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10-15,4,Normal Poll,2010-05-13
3,2010,May,14.0,16,19,Angus RS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10-15,5,Normal Poll,2010-05-14
4,2010,May,19.0,19,20,YouGov,Sun,NaN,NaN,NaN,...,Do you approve or disapprove of the Government...,NaN,NaN,NaN,NaN,NaN,10-15,6,Normal Poll,2010-05-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,2015,May,5.0,7,7,Populus,NaN,33.0,-1.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10-15,2092,Normal Poll,2015-05-05
2091,2015,May,6.0,6,7,ComRes,DM/ITV,35.0,0.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10-15,2093,Normal Poll,2015-05-06
2092,2015,May,6.0,6,<NA>,Survation,NaN,37.0,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10-15,2094,Normal Poll,2015-05-06
2094,2015,May,7.0,7,7,Exit poll,NaN,38.0,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10-15,2096,Exit Poll,NaT


In [221]:
sheet_name = xl.sheet_names[20]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
import datetime
## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


# Ashcroft - did the fieldwork really take over a month?


# # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published', 'Polling', 'Publisher',
         'Con', 'Unnamed: 8',
         'Lab', 'Unnamed: 10',
         'LD', 'Unnamed: 12',
         'UKIP', 'Unnamed: 14',
         'Green', 'Unnamed: 16',
         'Con lead', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
         'Con good', 'Con bad', 'Con net', 'Lab good', 'Lab bad',
         'Lab net', 'LD good', 'LD bad', 'LD net', 'UKIP good', 'UKIP bad',
         'UKIP net', 'Grn good', 'Grn bad', 'Grn net', 'Question', 'Con Best PM',
         'Lab Best PM', 'LD Best PM', 'UKIP Best PM', 'Question.1', 'Govt good',
         'Govt bad', 'Govt net', 'Question.2']]

        

df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day', 'Polling', 'Publisher',
              'Con', 'Con Error',
              'Lab', 'Lab Error',
              'LD', 'LD Error',
              'UKIP', 'UKIP Error',
              'Green', 'Green Error',
              'Con lead', 'Con lead Error', 'Notes 1', 'Notes 2',
            "Leader: Con good","Leader: Con bad","Leader: Con net",
            "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
            "Leader: LD good", "Leader: LD bad", "Leader: LD net",
            "Leader: UKIP good","Leader: UKIP bad","Leader: UKIP net",
            "Leader: Grn good","Leader: Grn bad","Leader: Grn net",
            "Leader: Question",
            "Con Best PM","Lab Best PM","LD Best PM","UKIP Best PM","Best PM Question",
            'Govt good', 'Govt bad',  'Govt net', 'Govt Question',

             ]
# #         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df["Fieldwork_Start_check_date"] = df["Unnamed: 3"]
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 2017GE = 8 June 2017
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2017,"Jun",8,8,8,"Result"]
# df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

15-17


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,UKIP Best PM,Best PM Question,Govt good,Govt bad,Govt net,Govt Question,sheet_name,sheet_row_number,Fieldwork_Start_check_date,Poll_Type
0,2015,May,8.0,9,10,Survation,MoS,40.0,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,15-17,2,2015-05-08,Normal Poll
1,2015,May,14.0,15,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,39.0,46.0,-7.0,Do you approve or disapprove of the Government...,15-17,3,2015-05-14,Normal Poll
2,2015,May,21.0,22,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,42.0,45.0,-3.0,Do you approve or disapprove of the Government...,15-17,4,2015-05-21,Normal Poll
3,2015,May,25.0,26,1,YouGov,Sun,41.0,NaN,29.0,...,NaN,NaN,39.0,44.0,-5.0,Do you approve or disapprove of the Government...,15-17,5,2015-05-25,Normal Poll
4,2015,May,28.0,29,31,YouGov,ST,NaN,NaN,NaN,...,NaN,NaN,40.0,44.0,-4.0,Do you approve or disapprove of the Government...,15-17,6,2015-05-28,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,2017,Jun,6.0,7,8,BMG,The Herald,46.0,NaN,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,15-17,324,2017-06-06,Normal Poll
323,2017,Jun,6.0,7,7,Survation,NaN,41.0,-1.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,15-17,325,2017-06-06,Normal Poll
324,2017,Jun,6.0,7,8,Ipsos MORI,ES,44.0,-1.0,36.0,...,NaN,Who do you think would make the most capable P...,NaN,NaN,NaN,NaN,15-17,326,2017-06-06,Normal Poll
325,2017,Jun,7.0,8,<NA>,Sky,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15-17,327,2017-06-07,Normal Poll


In [222]:
sheet_name = xl.sheet_names[21]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
import datetime
## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


# Ashcroft - did the fieldwork really take over a month?
# df.loc[df["Year"]=='?',["Year"]]= np.nan
df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
df.loc[df["Fieldwork"]=="5?",["Fieldwork"]]= "5"

df.loc[df["Published"]==datetime.datetime(2018, 12, 18, 0, 0),["Published"]]= "18"



# # # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published',
       'Polling','Publisher',
       'Con', 'Unnamed: 8', 'Lab', 'Unnamed: 10', 'LD',
       'Unnamed: 12', 'UKIP', 'Unnamed: 14', 'Green', 'Unnamed: 16', 'TIG/CUK',
       'Unnamed: 18', 'BXP', 'Unnamed: 20', 'Con lead', 'Unnamed: 22','Unnamed: 23', 'Unnamed: 24',
       'Con good', 'Con bad', 'Con net',
       'Lab good', 'Lab bad', 'Lab net',
       'LD good', 'LD bad', 'LD net',
       'UKIP good', 'UKIP bad', 'UKIP net',
       'Grn good', 'Grn bad', 'Grn net',
       'CUK good', 'CUK bad', 'CUK net',
       'BXP good', 'BXP bad', 'BXP net', 'Question',
       'Con Best PM', 'Lab Best PM', 'LD Best PM', 'BXP Best PM', 'Question.1',
       'Govt good', 'Govt bad', 'Govt net', 'Question.2']]

         

df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day',
              'Polling', 'Publisher',
              'Con', 'Con Error',
              'Lab', 'Lab Error',
              'LD', 'LD Error',
              'UKIP', 'UKIP Error',
              'Green', 'Green Error',
              'TIG/CUK', 'TIG/CUK Error',
              'BXP', 'BXP Error',
              'Con lead', 'Con lead Error', 'Notes 1', 'Notes 2',
            "Leader: Con good","Leader: Con bad","Leader: Con net",
            "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
            "Leader: LD good", "Leader: LD bad", "Leader: LD net",
            "Leader: UKIP good","Leader: UKIP bad","Leader: UKIP net",
            "Leader: Grn good","Leader: Grn bad","Leader: Grn net",
            "Leader: CUK good","Leader: CUK bad","Leader: CUK net",
            "Leader: BXP good","Leader: BXP bad","Leader: BXP net",              
            "Leader: Question",
            "Con Best PM","Lab Best PM","LD Best PM","BXP Best PM","Best PM Question",
            'Govt good', 'Govt bad',  'Govt net', 'Govt Question',

             ]
# #         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df["Fieldwork_Start_check_date"] = df["Unnamed: 3"]
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# 2019GE = 12 December 2019
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day","Poll_Type"]]=\
    [2019,"Dec",12,12,12,"Result"]
# df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

17-19


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,BXP Best PM,Best PM Question,Govt good,Govt bad,Govt net,Govt Question,sheet_name,sheet_row_number,Fieldwork_Start_check_date,Poll_Type
0,2017,Jun,9.0,10,11,YouGov,ST,NaN,NaN,NaN,...,NaN,Which of the following do you think would make...,NaN,NaN,NaN,NaN,17-19,2,2017-06-09,Normal Poll
1,2017,Jun,10.0,10,11,Survation,MoS,39.0,NaN,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,17-19,3,2017-06-10,Normal Poll
2,2017,Jun,11.0,12,15,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17-19,4,2017-06-11,Normal Poll
3,2017,Jun,15.0,16,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17-19,5,2017-06-15,Normal Poll
4,2017,Jun,16.0,17,19,Survation,GMB,41.0,NaN,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,17-19,6,2017-06-16,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,2019,Dec,10.0,11,11,Opinium,NaN,45.0,-1.0,33.0,...,NaN,"Which, if any, of the following people do you ...",NaN,NaN,NaN,NaN,17-19,564,2019-12-10,Normal Poll
563,2019,Dec,10.0,11,11,Opinium,NaN,NaN,NaN,NaN,...,6.0,"Which, if any, of the following people do you ...",NaN,NaN,NaN,NaN,17-19,565,2019-12-10,Normal Poll
564,2019,Dec,10.0,11,11,Panelbase,NaN,43.0,0.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,17-19,566,2019-12-10,Normal Poll
565,2019,Dec,10.0,11,11,Survation,NaN,44.5,0.0,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,17-19,567,2019-12-10,Normal Poll


In [223]:
sheet_name = xl.sheet_names[22]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
import datetime
## Manual Fix

df.loc[df["Published"]=="6/3",["Published"]]= "6"


##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


# Ashcroft - did the fieldwork really take over a month?
# df.loc[df["Year"]=='?',["Year"]]= np.nan
# df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
# df.loc[df["Fieldwork"]=="5?",["Fieldwork"]]= "5"


# df.loc[df["Published"]==datetime.datetime(2018, 12, 18, 0, 0),["Published"]]= "18"
# df.loc[df["Published"]==datetime.datetime(2012, 11, 21, 0, 0),["Published"]]= "21"
# df.loc[df["Published"]==datetime.datetime(2014, 1, 4, 0, 0),["Published"]]= "4"
# df.loc[df["Published"]==datetime.datetime(2013, 12, 26, 0, 0),["Published"]]= "26"

# # df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
# # df.loc[df["Date"]=='7?',["Date"]]= '7'
# # df.loc[df["Date"]=='19??',["Date"]]= '19'


# # df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# # df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # # # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published',
       'Polling', 'Publisher', 'Con', 'Unnamed: 8', 'Lab', 'Unnamed: 10', 'LD',
       'Unnamed: 12', 'Green', 'Unnamed: 14', 'BXP', 'Unnamed: 16', 'Ukip','Unnamed: 18', 'Con lead',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Con good', 'Con bad', 'Con net', 'Lab good', 'Lab bad', 'Lab net',
       'LD good', 'LD bad', 'LD net', 'BXP good', 'BXP bad', 'BXP net',
       'Grn good', 'Grn bad', 'Grn net', 'Question',
       'Con Best PM', 'Lab Best PM', 'LD Best PM', 'BXP Best PM', 'Question.1',
       'Govt good', 'Govt bad', 'Govt net', 'Question.2']]

        

df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day',
              'Polling', 'Publisher',
              'Con', 'Con Error',
              'Lab', 'Lab Error',
              'LD', 'LD Error',
              'Green', 'Green Error',
              'BXP', 'BXP Error',
              'UKIP', 'UKIP Error',
              'Con lead', 'Con lead Error', 'Notes 1', 'Notes 2',
            "Leader: Con good","Leader: Con bad","Leader: Con net",
            "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
            "Leader: LD good", "Leader: LD bad", "Leader: LD net",
            "Leader: BXP good","Leader: BXP bad","Leader: BXP net",                
            "Leader: Grn good","Leader: Grn bad","Leader: Grn net",
            "Leader: Question",
            "Con Best PM","Lab Best PM","LD Best PM","BXP Best PM","Best PM Question",
            'Govt good', 'Govt bad',  'Govt net', 'Govt Question',

             ]
# # #         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["sheet_name"] = sheet_name
df["sheet_row_number"] = df.index+2

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df["Fieldwork_Start_check_date"] = df["Unnamed: 3"]
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")
df["Poll_Type"] = "Normal Poll"
# # 2019GE = 12 December 2019
# df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
#     [2019,"Dec",12,12,12]
# # df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

19-


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,BXP Best PM,Best PM Question,Govt good,Govt bad,Govt net,Govt Question,sheet_name,sheet_row_number,Fieldwork_Start_check_date,Poll_Type
0,2019,Dec,13,14,17,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19-,2,2019-12-13,Normal Poll
1,2019,Dec,14,16,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,27.0,53.0,-26.0,Do you approve or disapprove of the Government...,19-,3,2019-12-14,Normal Poll
2,2019,Dec,21,23,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,34.0,46.0,-12.0,Do you approve or disapprove of the Government...,19-,4,2019-12-21,Normal Poll
3,2019,Dec,28,30,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,32.0,44.0,-12.0,Do you approve or disapprove of the Government...,19-,5,2019-12-28,Normal Poll
4,2020,Jan,4,6,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,30.0,45.0,-15.0,Do you approve or disapprove of the Government...,19-,6,2020-01-04,Normal Poll
5,2020,Jan,8,10,13,BMG,NaN,44.0,NaN,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,19-,7,2020-01-08,Normal Poll
6,2020,Jan,11,13,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,33.0,42.0,-9.0,Do you approve or disapprove of the Government...,19-,8,2020-01-11,Normal Poll
7,2020,Jan,15,17,18,Opinium,Ob,47.0,NaN,30.0,...,NaN,"Which, if any, of the following people do you ...",NaN,NaN,NaN,NaN,19-,9,2020-01-15,Normal Poll
8,2020,Jan,18,20,24,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,32.0,45.0,-13.0,Do you approve or disapprove of the Government...,19-,10,2020-01-18,Normal Poll
9,2020,Jan,24,26,28,YouGov,TT,49.0,NaN,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,19-,11,2020-01-24,Normal Poll


In [224]:
# df["Publication Day"][ df["Publication Day"].apply(lambda x: not isinstance(x,int)) ].dropna()
# df.loc[220 ,"Publication Day"]
# df["Fieldwork"][ df["Fieldwork"].apply(lambda x: True if isinstance(x,str) and "?" in x else False) ]
# df["Publication Day"][ df["Publication Day"].apply(lambda x: True if isinstance(x,str) and "/" in x else False) ]
# df["Year"][ df["Year"].apply(lambda x: True if isinstance(x,str) and "?" in x else False) ]

In [225]:
better_ordered_columns = \
['Year', 'Month', 'Fieldwork_Start_Day', 'Fieldwork_Finish_Day', 'Publication Day', 'Polling', 'Publisher',
    
'Con','Con Error', 'Con Female', 'Con Male', 'Con Male-Female',
'Lab','Lab Error', 'Lab Female', 'Lab Male', 'Lab Male-Female',
'LD', 'LD Error',  'LD Female',  'LD Male',  'LD Male-Female', 
'BNP',
'Green', 'Green Error', 
'UKIP', 'UKIP Error',
'BXP',  'BXP Error', 
'Referendum', 'Referendum Error',
'SDP','SDP Error',
'TIG/CUK', 'TIG/CUK Error',
 
'Con lead', 'Con lead Error',
 
'Leader: Question',
       'Leader: BXP bad', 'Leader: BXP good', 'Leader: BXP net',
       'Leader: CUK bad', 'Leader: CUK good', 'Leader: CUK net',
       'Leader: Con bad', 'Leader: Con good', 'Leader: Con net',
       'Leader: Grn bad', 'Leader: Grn good', 'Leader: Grn net',
       'Leader: LD bad', 'Leader: LD good', 'Leader: LD net',
       'Leader: Lab bad', 'Leader: Lab good', 'Leader: Lab net',
       'Leader: UKIP bad', 'Leader: UKIP good',
       'Leader: UKIP net',
 
'Govt Question', 'Govt bad', 'Govt good', 'Govt net',
 
'Best PM Question','Con Best PM','Lab Best PM','LD Best PM','BXP Best PM','UKIP Best PM',
 
'Pred Party Share: Question','Pred Party Share: Con','Pred Party Share: Lab','Pred Party Share: LD','Pred Party Share: UKIP',
 
'Notes 1','Notes 2',
'sheet_name','sheet_row_number','Poll_Type',
'Fieldwork_Start_check_date',
 ]

In [226]:
df_final = pd.concat(df_list,sort=False)[better_ordered_columns]
df_final.index = range(0,df_final.shape[0])


In [227]:
df_final["Notes 1"] = df_final["Notes 1"].apply(lambda x: x.strip() if pd.notnull(x) else np.nan)
df_final["Notes 2"] = df_final["Notes 2"].apply(lambda x: x.strip() if pd.notnull(x) else np.nan)

In [228]:
df_final.describe(include='all')

,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Con Female,...,Pred Party Share: Con,Pred Party Share: Lab,Pred Party Share: LD,Pred Party Share: UKIP,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type,Fieldwork_Start_check_date
count,6893.000000,6893,6155.0,6233.000000,4336.000000,6862,5447,6615.000000,6218.0,384.000000,...,25.000000,25.000000,25.000000,17.000000,3894,435,6890,6890.000000,6893,3711
unique,NaN,21,31.0,NaN,NaN,62,150,NaN,234.0,NaN,...,NaN,NaN,NaN,NaN,31,63,22,NaN,3,2605
top,NaN,Apr,11.0,NaN,NaN,YouGov,Sun,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,Internet,Certain to vote,10-15,NaN,Normal Poll,2010-04-23 00:00:00
freq,NaN,780,238.0,NaN,NaN,1885,971,NaN,1122.0,NaN,...,NaN,NaN,NaN,NaN,2932,143,2095,NaN,6858,9
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-05-19 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-01 00:00:00
mean,1999.652401,NaN,NaN,15.702872,15.897832,NaN,NaN,36.763401,NaN,39.539062,...,30.960000,33.160000,15.560000,13.294118,NaN,NaN,NaN,455.727576,NaN,NaN
std,17.723177,NaN,NaN,8.523298,8.790103,NaN,NaN,6.126637,NaN,4.377552,...,1.240967,2.867635,1.474223,1.862951,NaN,NaN,NaN,531.795301,NaN,NaN
min,1939.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,17.000000,NaN,27.000000,...,29.000000,27.000000,13.000000,10.000000,NaN,NaN,NaN,2.000000,NaN,NaN
25%,1989.000000,NaN,NaN,9.000000,8.000000,NaN,NaN,32.000000,NaN,37.000000,...,30.000000,32.000000,14.000000,12.000000,NaN,NaN,NaN,98.000000,NaN,NaN


In [229]:
df_final.columns

Index(['Year', 'Month', 'Fieldwork_Start_Day', 'Fieldwork_Finish_Day',
       'Publication Day', 'Polling', 'Publisher', 'Con', 'Con Error',
       'Con Female', 'Con Male', 'Con Male-Female', 'Lab', 'Lab Error',
       'Lab Female', 'Lab Male', 'Lab Male-Female', 'LD', 'LD Error',
       'LD Female', 'LD Male', 'LD Male-Female', 'BNP', 'Green', 'Green Error',
       'UKIP', 'UKIP Error', 'BXP', 'BXP Error', 'Referendum',
       'Referendum Error', 'SDP', 'SDP Error', 'TIG/CUK', 'TIG/CUK Error',
       'Con lead', 'Con lead Error', 'Leader: Question', 'Leader: BXP bad',
       'Leader: BXP good', 'Leader: BXP net', 'Leader: CUK bad',
       'Leader: CUK good', 'Leader: CUK net', 'Leader: Con bad',
       'Leader: Con good', 'Leader: Con net', 'Leader: Grn bad',
       'Leader: Grn good', 'Leader: Grn net', 'Leader: LD bad',
       'Leader: LD good', 'Leader: LD net', 'Leader: Lab bad',
       'Leader: Lab good', 'Leader: Lab net', 'Leader: UKIP bad',
       'Leader: UKIP good', 'Leader

In [230]:
df_final["Year"].value_counts()

2014    475
2013    419
2010    405
2012    383
2011    372
       ... 
1943      3
1946      2
1939      2
1944      1
1940      1
Name: Year, Length: 80, dtype: int64

In [231]:
df_final["Month"] = df_final["Month"].apply(lambda x: x.strip().replace("Sept","Sep").replace("July","Jul"))
df_final["Month"].value_counts()

Apr    782
May    657
Mar    643
Sep    618
Oct    610
Nov    584
Jun    551
Jan    550
Feb    533
Jul    477
Aug    465
Dec    423
Name: Month, dtype: int64

In [232]:
df_final

,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Con Female,...,Pred Party Share: Con,Pred Party Share: Lab,Pred Party Share: LD,Pred Party Share: UKIP,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type,Fieldwork_Start_check_date
0,1939,Feb,<NA>,<NA>,<NA>,Gallup,NaN,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DK 6%,NaN,NaN,Normal Poll,NaT
1,1939,Dec,<NA>,<NA>,<NA>,Gallup,NaN,54.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DK 16%,NaN,NaN,Normal Poll,NaT
2,1940,Feb,<NA>,<NA>,<NA>,Gallup,NaN,51.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DK 22%,NaN,NaN,Normal Poll,NaT
3,1943,Jun,<NA>,<NA>,<NA>,Gallup,News Chron,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,43-45,2.0,Normal Poll,NaT
4,1943,Jul,<NA>,<NA>,<NA>,Gallup,News Chron,33.0,-3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,43-45,3.0,Normal Poll,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6888,2020,Mar,26,27,29,Deltapoll,MoS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Internet,NaN,19-,39.0,Normal Poll,2020-03-26
6889,2020,Mar,26,28,29,Opinium,Ob,54.0,5,NaN,...,NaN,NaN,NaN,NaN,Internet,NaN,19-,40.0,Normal Poll,2020-03-26
6890,2020,Mar,28,30,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Internet,NaN,19-,41.0,Normal Poll,2020-03-28
6891,2020,Apr,1,2,3,YouGov,TT,52.0,4,NaN,...,NaN,NaN,NaN,NaN,Internet,NaN,19-,42.0,Normal Poll,2020-04-01


In [233]:
# df_final["Fieldwork_Start_Day"].value_counts().sort_index()
df_final["Fieldwork_Start_Day"] = df_final["Fieldwork_Start_Day"].astype('Int32').astype('float32')
df_final["Fieldwork_Start_Day"][df_final["Fieldwork_Start_Day"].notnull()].min(),df_final["Fieldwork_Start_Day"][df_final["Fieldwork_Start_Day"].notnull()].max()

(1.0, 31.0)

In [234]:
# df_final["Fieldwork_Finish_Day"].value_counts().sort_index()
df_final["Fieldwork_Finish_Day"][df_final["Fieldwork_Finish_Day"].notnull()].min(),df_final["Fieldwork_Finish_Day"][df_final["Fieldwork_Finish_Day"].notnull()].max()

(1, 31)

In [235]:
# df_final["Publication Day"].value_counts().sort_index()
df_final["Publication Day"][df_final["Publication Day"].notnull()].min(),df_final["Publication Day"][df_final["Publication Day"].notnull()].max()

(1, 31)

In [236]:
df_final["Polling"] = df_final["Polling"].replace("Result",np.nan).replace("Exit poll",np.nan)
df_final["Polling"].value_counts().head()

YouGov    1885
Gallup     895
MORI       676
ICM        608
NOP        551
Name: Polling, dtype: int64

In [237]:
df_final["Publisher"].value_counts().head(5)

Sun    971
DT     729
ST     520
TT     414
GU     347
Name: Publisher, dtype: int64

In [238]:
df_final["Con"].min(),df_final["Con"].max()

(17.0, 56.7)

In [239]:
df_final["Con Error"] = df_final["Con Error"].replace("-",np.nan)
df_final["Con Error"].min(),df_final["Con Error"].max(),
# [df_final["Con Error"][df_final["Con Error"].apply(lambda x:isinstance(x,str))].values[0]]

(-13.0, 15.0)

In [240]:
df_final["Con Female"].min(),df_final["Con Female"].max(),df_final["Con Male"].min(),df_final["Con Male"].max(),df_final['Con Male-Female'].min(),df_final['Con Male-Female'].max()

(27.0, 52.0, 28.0, 53.0, -13.0, 11.0)

In [241]:
df_final['Con Male-Female'].min(),df_final['Con Male-Female'].max(),

(-13.0, 11.0)

In [242]:
df_final["Lab"].min(),df_final["Lab"].max()

(18.0, 63.2)

In [243]:
df_final["Lab Female"].min(),df_final["Lab Female"].max(),df_final["Lab Male"].min(),df_final["Lab Male"].max(),df_final['Lab Male-Female'].min(),df_final['Lab Male-Female'].max()

(17.0, 47.0, 17.0, 46.0, -10.0, 9.0)

In [244]:
df_final["LD"].min(),df_final["LD"].max()

(1.5, 50.5)

In [245]:
df_final["LD Female"].min(),df_final["LD Female"].max(),df_final["LD Male"].min(),df_final["LD Male"].max(),df_final['LD Male-Female'].min(),df_final['LD Male-Female'].max()

(8.0, 36.0, 10.0, 35.0, -11.0, 9.0)

In [246]:
df_final["BNP"].min(),df_final["BNP"].max(),

(0.0, 5.0)

In [247]:
df_final["Green"].min(),df_final["Green"].max(),df_final["Green Error"].min(),df_final["Green Error"].max(),

(0.0, 13.0, -5.0, 7.0)

In [248]:
df_final["UKIP"].min(),df_final["UKIP"].max(),df_final["UKIP Error"].min(),df_final["UKIP Error"].max(),

(0.0, 25.0, -8.0, 5.0)

In [249]:
df_final["Referendum"].min(),df_final["Referendum"].max(),df_final["Referendum Error"].min(),df_final["Referendum Error"].max(),

(1.0, 4.0, -1.0, 1.0)

In [250]:
df_final["SDP"].min(),df_final["SDP"].max(),df_final["SDP Error"].min(),df_final["SDP Error"].max(),

(1.0, 11.0, -5.0, 6.0)

In [251]:
df_final["TIG/CUK"].min(),df_final["TIG/CUK"].max(),df_final["TIG/CUK Error"].min(),df_final["TIG/CUK Error"].max(),

(0.0, 18.0, -5.0, 4.0)

In [252]:
#'Con lead', 'Con lead Error',
# con-lab lead!
df_final["Con lead"].min(),df_final["Con lead"].max(),df_final["Con lead Error"].min(),df_final["Con lead Error"].max(),

(-43.5, 28.0, -29.0, 26.0)

In [253]:
df_final['Leader: Question'] = df_final['Leader: Question'].replace('To what extent do you approve or disapprove of the way ... is handling his job as ...','To what extent do you approve or disapprove of the way X is handling his job as Y')
df_final['Leader: Question'] = df_final['Leader: Question'].replace('Are you satisfied or dissatisfied with the way X is doing his job as Y?','Are you satisfied or dissatisfied with the way X is doing his/her job as…')

df_final['Leader: Question'].value_counts().head(20)

Do you think X is doing well or badly as… (LD leader)                                                            251
To what extent do you approve or disapprove of the way X is handling his job as Y                                182
Are you satisfied or dissatisfied with the way X is doing his/her job as…                                        108
Do you think (name) is doing well or badly as (position)?                                                         68
Do you have a favourable or unfavourable opinion of…                                                              59
Are you satisfied or dissatisified with the way X is doing his job as… (DPM)                                      58
Do you think (name) is or is not proving a good leader of X / Are you satisfied or dissatisfied with Z as PM?     46
Are you satisfied or dissatisfied with the way X is doing his job as…                                             31
Are you satisfied or disatisfied with the performance of X as Y?

In [254]:
# 'Leader: BXP bad',  'Leader: BXP good', 'Leader: BXP net',
df_final['Leader: BXP bad'].min(),df_final['Leader: BXP bad'].max(),df_final['Leader: BXP good'].min(),df_final['Leader: BXP good'].max(),df_final['Leader: BXP net'].min(),df_final['Leader: BXP net'].max(),

(37.0, 63.0, 19.0, 39.0, -36.0, 2.0)

In [255]:
# 'Leader: BXP bad', 'Leader: BXP good', 'Leader: BXP net',
# 'Leader: CUK bad', 'Leader: CUK good', 'Leader: CUK net',
# 'Leader: Con bad', 'Leader: Con good', 'Leader: Con net',
# 'Leader: Grn bad', 'Leader: Grn good', 'Leader: Grn net',
# 'Leader: LD bad',  'Leader: LD good',  'Leader: LD net',
# 'Leader: Lab bad', 'Leader: Lab good', 'Leader: Lab net',
# 'Leader: UKIP bad','Leader: UKIP good','Leader: UKIP net', 

df_final['Leader: CUK bad'].min(),\
    df_final['Leader: CUK bad'].max(),\
    df_final['Leader: CUK good'].min(),\
    df_final['Leader: CUK good'].max(),\
    df_final['Leader: CUK net'].min(),\
    df_final['Leader: CUK net'].max(),

(21.0, 35.0, 10.0, 14.0, -25.0, -10.0)

In [256]:
df_final['Leader: Con bad'].min(),\
    df_final['Leader: Con bad'].max(),\
    df_final['Leader: Con good'].min(),\
    df_final['Leader: Con good'].max(),\
    df_final['Leader: Con net'].min(),\
    df_final['Leader: Con net'].max(),

(12.0, 75.0, 18.0, 72.0, -56.0, 49.0)

In [257]:
df_final['Leader: Grn bad'].min(),\
    df_final['Leader: Grn bad'].max(),\
    df_final['Leader: Grn good'].min(),\
    df_final['Leader: Grn good'].max(),\
    df_final['Leader: Grn net'].min(),\
    df_final['Leader: Grn net'].max(),

(8.0, 46.0, 6.0, 32.0, -20.0, 9.0)

In [258]:
df_final['Leader: LD bad'].min(),\
    df_final['Leader: LD bad'].max(),\
    df_final['Leader: LD good'].min(),\
    df_final['Leader: LD good'].max(),\
    df_final['Leader: LD net'].min(),\
    df_final['Leader: LD net'].max(),

(9.0, 80.0, 5.0, 81.0, -65.0, 72.0)

In [259]:
df_final[['Leader: Lab bad','Leader: Lab good','Leader: Lab net']] = df_final[['Leader: Lab bad','Leader: Lab good','Leader: Lab net']].astype('float')
df_final[['Leader: Lab bad','Leader: Lab good','Leader: Lab net']].describe()

,Leader: Lab bad,Leader: Lab good,Leader: Lab net
count,980.000000,989.000000,1011.000000
mean,54.542857,28.210192,-25.700692
std,10.449216,7.442314,16.230917
min,17.000000,13.000000,-62.000000
25%,48.000000,23.000000,-36.500000
50%,56.000000,27.000000,-27.000000
75%,62.000000,32.000000,-17.000000
max,78.000000,65.000000,48.000000


In [260]:
df_final['Leader: UKIP bad'].min(),\
df_final['Leader: UKIP bad'].max(),\
df_final['Leader: UKIP good'].min(),\
df_final['Leader: UKIP good'].max(),\
df_final['Leader: UKIP net'].min(),\
df_final['Leader: UKIP net'].max(),

(18.0, 64.0, 3.0, 61.0, -56.0, 35.0)

In [261]:
# 'Govt Question',
# 'Govt bad', 'Govt good', 'Govt net',
df_final["Govt Question"].value_counts()

Do you approve or disapprove of the Government's record to date?                                              1275
Are you satisfied or dissatisfied with the way the Govt is running the country?                                105
From what you have seen or heard do you think ... Is doing a good job or a bad job?                             11
Are you satisfied or dissatisfied with the Government's record to date?                                          8
Do you approve or disapprove of the way the Govt is running the country?                                         5
From what you have heard do you think ... Is doing a good job or a bad job?                                      3
(Q not asked.)                                                                                                   1
Do you approve or disapprove of the government's record to date?                                                 1
Taken as a whole, do you approve or disapprove of the government's record since 

In [262]:
df_final['Govt bad'].min(),df_final['Govt bad'].max(),\
df_final['Govt good'].min(),df_final['Govt good'].max(),\
df_final['Govt net'].min(),df_final['Govt net'].max()\

(22.0, 86.0, 8.0, 73.0, -77.0, 49.0)

In [263]:
# 'Best PM Question',
# 'Con Best PM', 'Lab Best PM', 'LD Best PM', 'BXP Best PM', 'UKIP Best PM',

df_final['Best PM Question'].value_counts().head(10)

Which of the following do you think would make the best PM?                                                               125
Which, if any, of the following people do you think would be the best prime minister?                                     100
If you had to choose between the two, who would you prefer to see as the next Prime Minister?                              33
Which of the following party leaders do you think would make the best PM?                                                   8
If you had to choose between X and Y, who do you think would m ake the best leader for Britain?                             7
Who do you think would make the most capapble PM?                                                                           7
If you had to choose between Boris Johnson and Jeremy Corbyn, who do you think would make the best leader for Britain?      5
Who do you think would make the most capable Prime Minister?                                                          

In [264]:
df_final['Con Best PM'].min(),df_final['Con Best PM'].max(),\
df_final['Lab Best PM'].min(),df_final['Lab Best PM'].max(),\
df_final['LD Best PM'].min(), df_final['LD Best PM'].max(),\
df_final['BXP Best PM'].min(),df_final['BXP Best PM'].max(),\
df_final['UKIP Best PM'].min(), df_final['UKIP Best PM'].max(),\

(15.0, 67.0, 12.0, 43.0, 3.0, 21.0, 6.0, 11.0, 1.0, 12.0)

In [265]:
# 'Pred Party Share: Question', 'Pred Party Share: Con',
#        'Pred Party Share: Lab', 'Pred Party Share: LD',
#        'Pred Party Share: UKIP',
        
df_final['Pred Party Share: Question'].value_counts()

What % share do you think party would win...    25
Name: Pred Party Share: Question, dtype: int64

In [266]:
df_final[['Pred Party Share: Question', 'Pred Party Share: Con',
       'Pred Party Share: Lab', 'Pred Party Share: LD',
       'Pred Party Share: UKIP',]].describe()


,Pred Party Share: Con,Pred Party Share: Lab,Pred Party Share: LD,Pred Party Share: UKIP
count,25.000000,25.000000,25.000000,17.000000
mean,30.960000,33.160000,15.560000,13.294118
std,1.240967,2.867635,1.474223,1.862951
min,29.000000,27.000000,13.000000,10.000000
25%,30.000000,32.000000,14.000000,12.000000
50%,31.000000,32.000000,16.000000,13.000000
75%,31.000000,36.000000,16.000000,15.000000
max,35.000000,39.000000,18.000000,17.000000


In [267]:
df_final.dtypes.value_counts()

float64           64
object            11
Int64              2
int32              1
datetime64[ns]     1
float32            1
dtype: int64

In [268]:
df_final[df_final.columns[df_final.dtypes=='object']]

,Month,Polling,Publisher,Leader: Question,Govt Question,Best PM Question,Pred Party Share: Question,Notes 1,Notes 2,sheet_name,Poll_Type
0,Feb,Gallup,NaN,NaN,NaN,NaN,NaN,NaN,DK 6%,NaN,Normal Poll
1,Dec,Gallup,NaN,NaN,NaN,NaN,NaN,NaN,DK 16%,NaN,Normal Poll
2,Feb,Gallup,NaN,NaN,NaN,NaN,NaN,NaN,DK 22%,NaN,Normal Poll
3,Jun,Gallup,News Chron,NaN,NaN,NaN,NaN,NaN,NaN,43-45,Normal Poll
4,Jul,Gallup,News Chron,NaN,NaN,NaN,NaN,NaN,NaN,43-45,Normal Poll
...,...,...,...,...,...,...,...,...,...,...,...
6888,Mar,Deltapoll,MoS,Do you think they are doing very well in their...,NaN,NaN,NaN,Internet,NaN,19-,Normal Poll
6889,Mar,Opinium,Ob,To what extent do you approve or disapprove of...,NaN,"Which, if any, of the following people do you ...",NaN,Internet,NaN,19-,Normal Poll
6890,Mar,YouGov,NaN,NaN,Do you approve or disapprove of the Government...,NaN,NaN,Internet,NaN,19-,Normal Poll
6891,Apr,YouGov,TT,Do you have a favourable or unfavourable opini...,NaN,NaN,NaN,Internet,NaN,19-,Normal Poll


In [269]:
df_final[["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day"]]

,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day
0,1939,Feb,NaN,<NA>
1,1939,Dec,NaN,<NA>
2,1940,Feb,NaN,<NA>
3,1943,Jun,NaN,<NA>
4,1943,Jul,NaN,<NA>
...,...,...,...,...
6888,2020,Mar,26.0,27
6889,2020,Mar,26.0,28
6890,2020,Mar,28.0,30
6891,2020,Apr,1.0,2


In [270]:
start_before_finish = ((df_final["Fieldwork_Start_Day"]>df_final["Fieldwork_Finish_Day"]) & df_final["Fieldwork_Start_Day"].notnull() & df_final["Fieldwork_Finish_Day"].notnull())

In [271]:
start_before_finish.sum()

397

In [272]:
df_final.loc[start_before_finish]

,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Con Female,...,Pred Party Share: Con,Pred Party Share: Lab,Pred Party Share: LD,Pred Party Share: UKIP,Notes 1,Notes 2,sheet_name,sheet_row_number,Poll_Type,Fieldwork_Start_check_date
144,1955,Dec,24.0,4,<NA>,Gallup,News Chron,44.5,-2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55-59,7.0,Normal Poll,NaT
148,1956,Apr,24.0,4,<NA>,Gallup,News Chron,44.5,0.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55-59,11.0,Normal Poll,NaT
163,1957,Jun,24.0,1,<NA>,Gallup,News Chron,41.5,0.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55-59,26.0,Normal Poll,NaT
167,1957,Sep,29.0,10,<NA>,Gallup,News Chron,40.5,-1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55-59,30.0,Normal Poll,NaT
178,1958,May,30.0,6,<NA>,Gallup,News Chron,39.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55-59,41.0,Normal Poll,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6821,2019,Nov,30.0,1,<NA>,YouGov,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Internet,NaN,17-19,539.0,Normal Poll,2019-11-30
6864,2020,Jan,31.0,2,5,YouGov,TT,49.0,0.0,NaN,...,NaN,NaN,NaN,NaN,Internet,NaN,19-,15.0,Normal Poll,2020-01-31
6865,2020,Jan,31.0,3,6,Ipsos MORI,ES,47.0,NaN,NaN,...,NaN,NaN,NaN,NaN,Internet,NaN,19-,16.0,Normal Poll,2020-01-31
6875,2020,Feb,28.0,2,12,Ipsos MORI,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Internet,NaN,19-,26.0,Normal Poll,2020-02-28


In [273]:
df_final["Fieldwork_Start_Day"].mean().round(),df_final["Fieldwork_Finish_Day"].mean().round()

(15.0, 16.0)

In [274]:
df_final["Fieldwork_Start_Date"] = df_final["Fieldwork_Start_Day"]
df_final["Fieldwork_Start_Date"] = df_final["Fieldwork_Start_Date"].fillna(df_final["Fieldwork_Start_Day"].mean().round())
df_final["Fieldwork_Start_Date"] = df_final["Fieldwork_Start_Date"].astype(int).astype(str).apply(lambda x: x.zfill(2))
df_final["Fieldwork_Finish_Date"] = df_final["Fieldwork_Finish_Day"]
df_final["Fieldwork_Finish_Date"] = df_final["Fieldwork_Finish_Date"].fillna(df_final["Fieldwork_Finish_Day"].mean().round())
df_final["Fieldwork_Finish_Date"] = df_final["Fieldwork_Finish_Date"].astype(int).astype(str).apply(lambda x: x.zfill(2))

In [275]:
df_final["Fieldwork_Start_Date"] = df_final[["Year","Month","Fieldwork_Start_Date"]].astype('str').apply(lambda x: "".join(x),axis=1)\
    .apply(lambda x:pd.to_datetime(x,format="%Y%b%d"))
df_final["Fieldwork_Finish_Date"] = df_final[["Year","Month","Fieldwork_Finish_Date"]].astype('str').apply(lambda x: "".join(x),axis=1)\
    .apply(lambda x:pd.to_datetime(x,format="%Y%b%d"))

In [276]:
df_final[["Fieldwork_Start_Date","Fieldwork_Start_check_date"]].dropna()

# df_final["Date Inconsistency"] = False
# df_final.loc[df_final[["Fieldwork_Start_Date","Fieldwork_Start_check_date"]].dropna().index,"Date Inconsistency"] = True

,Fieldwork_Start_Date,Fieldwork_Start_check_date
3173,2005-05-19,2005-05-19
3174,2005-05-19,2005-05-19
3175,2005-05-24,2005-05-24
3176,2005-06-16,2005-06-16
3177,2005-06-17,2005-06-17
...,...,...
6888,2020-03-26,2020-03-26
6889,2020-03-26,2020-03-26
6890,2020-03-28,2020-03-28
6891,2020-04-01,2020-04-01


In [277]:
# df_final[["Fieldwork_Start_Date","Fieldwork_Start_check_date"]][df_final["Date Inconsistency"]==True]

In [278]:
mask = df_final["Fieldwork_Start_check_date"].notnull()&df_final["Fieldwork_Start_Date"].notnull()
df_final[mask][ df_final["Fieldwork_Start_Date"][mask]!= df_final["Fieldwork_Start_check_date"][mask] ]

df_final["Date Inconsistency"] = False
df_final.loc[df_final[mask][ df_final["Fieldwork_Start_Date"][mask]!= df_final["Fieldwork_Start_check_date"][mask] ].index,"Date Inconsistency"] = True
### NEEDS FIXING!

In [279]:
# df_final[ df_final[["Year","Month","Fieldwork_Start_Date"]].astype('str').apply(lambda x: "".join(x),axis=1)=='2006Sep31' ]

In [280]:
# import calendar
# month_replace_dict = {calendar.month_abbr[x]:str(x).zfill(2) for x in range(1,len(calendar.month_abbr))}

# df_final["Fieldwork_Start_Date"].head().apply(lambda x:x.replace(month_replace_dict))

In [281]:
# adjust for entries where Fieldwork Finish Date is in *next* month

df_final.loc[start_before_finish,"Fieldwork_Finish_Date"] = df_final.loc[start_before_finish,"Fieldwork_Finish_Date"] +pd.DateOffset(months= 1)

In [282]:
df_final["Fieldwork_Midpoint_Date"] = df_final["Fieldwork_Start_Date"]+((df_final["Fieldwork_Start_Date"] - df_final["Fieldwork_Start_Date"])/2)

In [283]:
df_final.drop("Date Inconsistency",axis=1,inplace=True)

In [284]:
df_final.drop("Fieldwork_Start_check_date",axis=1,inplace=True)

In [285]:
df_final[df_final.columns[df_final.dtypes=='Int64']] = df_final[df_final.columns[df_final.dtypes=='Int64']].astype('float32')

In [286]:
df_final.dtypes.value_counts()

float64           64
object            11
datetime64[ns]     3
float32            3
int32              1
dtype: int64

In [287]:
{x:x.replace("Error","Change") for x in df_final if "Error" in x}

{'Con Error': 'Con Change',
 'Lab Error': 'Lab Change',
 'LD Error': 'LD Change',
 'Green Error': 'Green Change',
 'UKIP Error': 'UKIP Change',
 'BXP Error': 'BXP Change',
 'Referendum Error': 'Referendum Change',
 'SDP Error': 'SDP Change',
 'TIG/CUK Error': 'TIG/CUK Change',
 'Con lead Error': 'Con lead Change'}

In [288]:
df_final = df_final.rename(columns={x:x.replace("Error","Change") for x in df_final if "Error" in x})

In [289]:
df_final.to_csv(Pollbase_folder+"MarkPackPollBase_flat_v7.csv")

# 2020 version

# exit polls fixed, notes 1 & notes 2 added
# Full Fieldwork_Start/Finish_Dates built (missing day value inferred as mean of filled in start/finish day - 15/16)
# Fieldwork_Midpoint_Date created
# Initial three polls added
# Date Inconsistency added

# Would be really trivial to add proper "publication date"